# Design

# LORA Modules

## Building a gateway
The main problem with a gateway is the sx130x-chip requirement. (https://www.thethingsnetwork.org/forum/t/creating-a-lorawan-gatway-from-esp32/66580) since the lorawan specifications require at least 8 channels with concurrent demodulation support and concurrent message handling is possible.

## LoRa module types
LoRa is patented by semtech. That is the chirp signal modulation and demodulation.
### Semtech
Under the semtech-denominator there are a couple of modules integrating the semtech-LoRa modules: \
-> SX1276, SX1278 \
Such as:
-> Microchip R \
### Microchip
-> RN2483

### STM32
-> STM32WL-series, worked out through cooperation with semtech based on the sx126x semtech-series

### HopeRF
low-cost LoRa modules.

### RAKWireless
LoRaWan module based on Semtech S1276

## DL-LLCC68-S
A bit unclear, but it seems like these modules cover a spectrum from 150 - 960 MHz.

**Sources:**
> https://patents.google.com/patent/US8406275B2/en \
ElektorMag - LoRaWan options
> https://sps.mesago.com/content/dam/messefrankfurt-mesago/sps/2023/documents/pdf/en/EN2023011_compressed.pdf

